### Step 4: Define a new score!

The code shown below assumes that we have a database, which contains all the results of the conjunctive query, without an order.

In [1]:
import pandas as pd

To define the new score, we use the following variables:
1. average_rate_per_night
2. bedrooms_count	
3. city
4. date_of_listing

Following we explain the functions that were used to compute the score, in each of the variables

#### 1. Average rate per night: 

This function compares the ideal rate for the user, which is explicitly asked, and the actual rate per nigth of each Airbnb, called by the variable doc_rate. Resulting in a rate between 0 and 1, called rate_score.

- The maximum rate_score you can have is 1, and it is reached when user_rate = doc_rate.
- The minimum rate_score you can have is 0, and it is reached when doc_rate it's more than 30 dollars away from user_rate
- The intermediate values are given by areas around our variable of interest: user_rate, giving more points to values closer and less to the farthest.

In [2]:
def rate_score(user_rate, doc_rate):
    if user_rate == doc_rate:
        rate_score = 1
    elif user_rate - 10 <= doc_rate <= user_rate + 10:
        rate_score = 0.75
    elif user_rate - 20 <= doc_rate <= user_rate + 20:
        rate_score = 0.5
    elif user_rate - 30 <= doc_rate <= user_rate + 30:
        rate_score = 0.25
    else:
        rate_score = 0
    return rate_score

In [3]:
rate_score(25, 45)

0.5

In [ ]:
# Remove the $ 
df['average_rate_per_night'] = df['average_rate_per_night'].str.replace("$", '')
# Convert to integer
df['average_rate_per_night'] = pd.to_numeric(df['average_rate_per_night'])

In order to execute the previous function, we need to make some small modifications to the data as removing the dollar sign and convert the variable to an integer.

#### 2. Bedrooms count:

This function compares the number of bedrooms required by the user, which is explicitly asked, and the actual number of bedrooms of each Airbnb, called by the variable doc_bedroom. Resulting in a rate between 0 and 1, called bedroom_score.

- The maximum bedroom_score you can have is 1, and it is reached when user_bedroom = doc_bedroom.
- The minimum bedroom_score you can have is 0, and it is reached when doc_bedroom it's more than 3 bedrooms away from user_bedroom.
- The intermediate values are given by areas around our variable of interest: user_bedrrom, giving more points to values closer and less to the farthest.

In [209]:
def bedroom_score(user_bedroom, doc_bedroom):
    if user_bedroom == doc_bedroom:
        bedroom_score = 1
    elif user_bedroom - 1 <= doc_bedroom <= user_bedroom + 1:
        bedroom_score = 0.75
    elif user_bedroom - 2 <= doc_bedroom <= user_bedroom + 2:
        bedroom_score = 0.5
    elif user_bedroom - 3 <= doc_bedroom <= user_bedroom + 3:
        bedroom_score = 0.25
    else:
        bedroom_score = 0
    return bedroom_score

In order to execute the previous function, we need to make some small modifications to the data as removing null values. In this case, the variable bedrooms_count contains numbers, and the string "Studio". A Studio is a small apartment which combines living room, bedroom, and kitchen into a single room, for that reason, we assume that when it says "Studio", it refers to a single bedroom. Ultimately we convert the variable to an integer.

In [124]:
# Remove null values
df = df[pd.notnull(df["bedrooms_count"])]
# Replace Studio for 1
df['bedrooms_count'] = df['bedrooms_count'].str.replace('Studio', '1')
# Convert to integer
df['bedrooms_count'] = df['bedrooms_count'].astype(int)

#### 3. City: 

This function compares the city preference specified by the user, and the actual city of each Airbnb, called by the variable doc_city. Resulting in a binary rate that takes the value 1 when user_city = doc_city, and 0 otherwise. 

In [ ]:
pip install geopy

In [1]:
import geopy.distance

ModuleNotFoundError: No module named 'geopy'

In [211]:
def city_score(user_city, doc_city):
    coords_1 = (user_city_latitude, user_city_longitude) #we have to found a way to get the coordinates from the name of the city
    coords_2 = (doc_city_latitude, doc_city_longitude)
    distance = geopy.distance.vincenty(coords_1, coords_2).km
    if 0 < distance < 5:
        city_score = 1
    elif 5 =< distance <= 10:
        city_score = 0.50
    elif 10 <= distance <= 20:
        city_score = 0.25
    else:
        city_score = 0.10
    return city_score

#### 4. Date of listing: 

In this case, the score assigned to each Airbnb, is not given by the information provided by the user. 
We assume that an older date of listing means that the owner of the Airbnb is a more experienced one and therefore is a better option for the user. In this way, we give a lower score to the newer Airbnb and a higher one to the older ones. The date_score variable also belongs to the interval between 0 and 1. The chosen method is the follower: we calculate the quantiles of the dates' distribution and we assign a score based on the date position: the oldest dates'll be near the first quantile, the yongest dates 'll be near the third quantile. This method always guarantees a higher score on the oldest dates.

In [ ]:
def date_score(doc_date_of_listing):
    df.date_of_listing = pd.to_datetime(df.date_of_listing)
    first_quantile=df.date_of_listing.quantile(0.25)
    median = df.date_of_listing.quantile(0.5)
    third_quantile = df.date_of_listing.quantile(0.75)
    
    if doc_date_of_listing <= first_quantile:
        date_score = 1
    elif first_quantile < doc_date_of_listing <= median:
        date_score = 0.75
    elif median < doc_date_of_listing < third_quantile:
        date_score = 0.5
    elif doc_date_of_listing >= third_quantile:
        date_score = 0.25
    
    return date_score

####  Final Score
To finish, we will define the function that calculates the final score for each Airbnb, doing a weighted average of our four previously calculated scores. As we do not find any relevant reason to assign more weight to one variable than to another, we weight each sub score, as 25% of the total.

Again we have a result that oscillates between 0 and 1, where 1 represents a greater similarity of Airbnb with the preferences of the user.

In [213]:
def final_score(rate_score, bedroom_score, city_score, date_score):
    return 0.25*rate_score + 0.25*bedroom_score + 0.25*city_score + 0.25*date_score

### EXAMPLE

Let's the whole df dataframe is the result of the conjunctive query that we have to rank

In [201]:
# Read the database
df = pd.read_csv("Airbnb_Texas_Rentals.csv")
# Select only the first 5 rows and name it cqres
# Remove the $ 
df['average_rate_per_night'] = df['average_rate_per_night'].str.replace("$", '')
# Convert to integer
df['average_rate_per_night'] = pd.to_numeric(df['average_rate_per_night'])
# Remove null values
df = df[pd.notnull(df["bedrooms_count"])]
# Replace Studio for 1
df['bedrooms_count'] = df['bedrooms_count'].str.replace('Studio', '1')
# Convert to integer
df['bedrooms_count'] = df['bedrooms_count'].astype(int)

In [215]:
df = df.iloc[:5]

In [216]:
df

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,New Score
0,1,27.0,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,0.4375
1,2,149.0,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,0.1875
2,3,59.0,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,0.5
3,4,60.0,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,0.5
4,5,75.0,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,0.375


On the other hand, we need gather information about user's preferences

In [217]:
user_rate = int(input("How much is the ideal rate you would pay per night? "))
user_bedroom = int(input("How many bedrooms do you need? "))
user_city = input("In wich city do you prefer to stay? ")

How much is the ideal rate you would pay per night? 56
How many bedrooms do you need? 2
In wich city do you prefer to stay? Dallas


In [218]:
# create a new column with empty values
df["New Score"] = ""

In [225]:
# fill that empty space with the result of computing the final score
for i in range(len(df)):
    rs = rate_score(user_rate, df["average_rate_per_night"][i])
    bs = bedroom_score(user_bedroom, df["bedrooms_count"][i])
    cs = city_score(user_city, df["city"][i])
    ds = date_score(doc_date_of_listing, df["date_of_listing"][i])
    fs = final_score(rs, bs, cs, ds)
    df["New Score"][i] = fs

C:\Users\falli\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [236]:
# Sort the values by score in descending order
df = df.sort_values('New Score', ascending=False)
# Reset the index, using now the sorted values
df = df.reset_index(drop=True)
# Make the index start from 1
df.index += 1 
# Rename the index as Ranking
df.index.name = 'Ranking'
# Select only the columns of interest and show the results
cols_of_interes = ["title", "description", "city", "url"]
df[cols_of_interes]

,title,description,city,url
Ranking,,,,
1,River house near the city,'River house on island close to the city' \nA ...,Houston,https://www.airbnb.com/rooms/16926307?location...
2,Private Room Close to Campus,Private bedroom in a cute little home situated...,Bryan,https://www.airbnb.com/rooms/11839729?location...
3,The Porch,Welcome to our original 1920's home. We recent...,Fort Worth,https://www.airbnb.com/rooms/17325114?location...
4,2 Private rooms/bathroom 10min from IAH airport,Welcome to stay in private room with queen bed...,Humble,https://www.airbnb.com/rooms/18520444?location...
5,Unique Location! Alamo Heights - Designer Insp...,"Stylish, fully remodeled home in upscale NW – ...",San Antonio,https://www.airbnb.com/rooms/17481455?location...
